## PyTorch Tutorial #17 - Save and Load Models

In [1]:
import torch
import torch.nn as nn

c:\Users\Leandro\source\repos\PyTorchTest\PyTorchTest\TorchEnv\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class Model(nn.Module):
    def __init__(self, n_input_features):
        super(Model, self).__init__()
        self.linear = nn.Linear(n_input_features, 1)

    def forward(self, x):
        y_pred = torch.sigmoid(self.linear(x))
        return y_pred

In [20]:
model = Model(n_input_features = 6)

### Manera LAZY de guardar un modelo

In [21]:
# Guardo modelo.
FILE = 'model.pth'
torch.save(model, FILE)

In [22]:
# Cargo modelo.
model = torch.load(FILE)
# Luego de cargarlo SIEMPRE lo pongo en eval.
model.eval()

Model(
  (linear): Linear(in_features=6, out_features=1, bias=True)
)

In [23]:
# Veo los parámetros del modelo para ver si se cargó correctamente.
for param in model.parameters():
    print(param)

Parameter containing:
tensor([[-0.2252, -0.0574, -0.2888, -0.2559, -0.3939, -0.1680]],
       requires_grad=True)
Parameter containing:
tensor([-0.2812], requires_grad=True)


### Manera PREFERIBLE de guardar un modelo

In [24]:
# Guardo el state_dict del modelo.
torch.save(model.state_dict(), FILE)

In [25]:
loaded_model = Model(n_input_features=6)
loaded_model.load_state_dict(torch.load(FILE))
loaded_model.eval()

Model(
  (linear): Linear(in_features=6, out_features=1, bias=True)
)

In [26]:
# Veo los parámetros del modelo para ver si se cargó correctamente.
for param in loaded_model.parameters():
    print(param)

Parameter containing:
tensor([[-0.2252, -0.0574, -0.2888, -0.2559, -0.3939, -0.1680]],
       requires_grad=True)
Parameter containing:
tensor([-0.2812], requires_grad=True)


In [27]:
# El state_dict del modelo.
model.state_dict()

OrderedDict([('linear.weight',
              tensor([[-0.2252, -0.0574, -0.2888, -0.2559, -0.3939, -0.1680]])),
             ('linear.bias', tensor([-0.2812]))])

### Para crear un checkpoint y reanudar el entrenamiento de un modelo

In [31]:
# Cuando tengo un modelo completo con learning_rate y optimizer, el optimizer también tiene su state_dict.
learning_rate = 0.01
optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate)
optimizer.state_dict()

{'state': {},
 'param_groups': [{'lr': 0.01,
   'momentum': 0,
   'dampening': 0,
   'weight_decay': 0,
   'nesterov': False,
   'maximize': False,
   'foreach': None,
   'differentiable': False,
   'params': [0, 1]}]}

In [34]:
# Necesito crear un diccionario con todos los elementos que deseo cargar.
checkpoint = {
    'epoch': 90,
    'model_state': model.state_dict(),
    'optim_state': optimizer.state_dict()
}

In [35]:
# Guardo el estado actual del modelo.
torch.save(checkpoint, 'checkpoint.pth')

In [36]:
# Cuando quiero cargarlo y reanudar le entrenamiento, primero necesito 
# crear el modelo y optimizador con las mismas características, luego les cargo el state_dict.
model = Model(n_input_features=6)
optimizer = torch.optim.SGD(model.parameters(), lr = 0)

# Cargo el checkpoint.
loaded_checkpoint = torch.load('checkpoint.pth')

# Cargo cada uno de los elementos del diccionario.
epoch = loaded_checkpoint['epoch']
model.load_state_dict(checkpoint['model_state'])
optimizer.load_state_dict(checkpoint['optim_state'])

### Cuando se utiliza GPU

In [ ]:
# En el caso en que se guarde el modelo en la GPU y se cargue en la CPU.
device = torch.device('cuda')
model.to(device)
torch.save(model.state_dict(), PATH)

# Ahora lo quiero cargar en la CPU
device = torch.device('cpu')
model = Model(*args, **kwargs)
model.load_state_dict(torch.load(PATH, map_location = device))

In [ ]:
# En el caso en que se guarde el modelo en la GPU y se cargue en la GPU.
device = torch.device('cuda')
model.to(device)
torch.save(model.state_dict(), PATH)

# Ahora lo quiero cargar en la GPU
model = Model(*args, **kwargs)
model.load_state_dict(torch.load(PATH))
model.to(device)

In [ ]:
# En el caso en que se guarde el modelo en la CPU y se cargue en la GPU.
torch.save(model.state_dict(), PATH)

# Ahora lo quiero cargar en la GPU
device = torch.device('cuda')
model = Model(*args, **kwargs)
model.load_state_dict(torch.load(PATH, map_location = 'cuda:0'))
model.to(device)